# Natural Language Processing Mini Task

**Author:** Ties de Kok ([Personal Website](http://www.tiesdekok.com))  
**Last updated:** 15 May 2018  
**Python version:** Python 3.6  
**License:** MIT License  
**Credit:** part of these tasks were co-created by Stephan Hollander ([Personal Website](https://www.tilburguniversity.edu/webwijs/show/s.hollander/))

## *Introduction*

In this notebook I will provide you with "tasks" that you can try to solve.  

Most of what you need is discussed in the tutorial notebooks, the rest you will have to Google (which is an important exercise in itself).

## *Relevant notebooks*

1) [`0_python_basics.ipynb`](https://nbviewer.jupyter.org/github/TiesdeKok/LearnPythonforResearch/blob/master/0_python_basics.ipynb)  


2) [`2_handling_data.ipynb`](https://nbviewer.jupyter.org/github/TiesdeKok/LearnPythonforResearch/blob/master/2_handling_data.ipynb)  


3) [`NLP_Notebook.ipynb`](https://nbviewer.jupyter.org/github/TiesdeKok/Python_NLP_Tutorial/blob/master/NLP_Notebook.ipynb)  

## NLP Mini Task <br> --------------------

The goal of this mini-task is to get hands-on experience with handling, cleaning, and analyzing textual data using Python.

---------------------------------------------------

There are two primary tasks and one challenging bonus task:

**Task 1)** Following Garcia and Norli (2012), extract state name counts from MD&As to assess geographic dispersion  

**Task 2)** Create a sentiment score for MD&As based on the Loughran and McDonald (2011) word lists  

---------------------------------------------------

**Bonus task 3)** Combine task 1 and 2, evaluate the sentiment score relating to state name references

#### References  

Garcia, D., & Norli, Ø. (2012). Geographic dispersion and stock returns. Journal of Financial Economics, 106(3), 547-565.  
Loughran, T., & McDonald, B. (2011). When is a liability not a liability? Textual analysis, dictionaries, and 10‐Ks. The Journal of Finance, 66(1), 35-65.

### Data

Gathering and extracting the MD&A section of a 10-K is quite tricky.  

I have therefore included a random selection of 20 pre-processed MDA filings.  

In the "data" folder you will find a folder called "MDA_files". Each file in this folder is an MD&A filing, the filename is the unique identifier.

You will also find a file called `MDA_META_DF.xlsx` in the "data" folder, this contains the following meta-data for eaching MD&A: (filing date, cik, company name, and link to filing)

### Import required packages

In [1]:
import os, re
import pandas as pd
import numpy as np
from os.path import join

In [2]:
from tqdm import tqdm_notebook as tqdm

In [3]:
import en_core_web_sm
nlp = en_core_web_sm.load()

### Load data

The files should all be in the following folder:  
```
join('data', 'MDA_files')
```

In [4]:
mda_file_folder = join('data', 'MDA_files')
mda_files = [file for file in os.listdir(mda_file_folder) if file[-3:] == 'txt']

In [5]:
mda_data = {}
for mda_file in mda_files:
    with open(join(mda_file_folder, mda_file), 'r') as f:
        mda_data[mda_file] = f.read()

### Clean and Pre-process text data

You might need to split it into sentences, maybe split it into words, maybe remove invalid characters.

Whatever you see fit.

----
Split into sentences

In [6]:
mda_sentences = {}
for key, value in tqdm(mda_data.items()):
    mda_sentences[key] = list(nlp(value).sents)

HBox(children=(IntProgress(value=0, max=20), HTML(value='')))

### Task 1: Extract state name counts

Follow Garcia and Norli (2012) and count the number of times that each U.S. state name is mentioned in the MD&A.  

Then:

1. Create a DataFrame for each MD&A that shows the number of times each U.S. state name is mentioned.  
2. Create a DataFrame to report the min, max, mean, median, and stdev for the number of times that each state is mentioned in MD&As. 

**Note:** state names are provided in the `state_names.xlsx` file in the "data" folder.

In [7]:
state_names_df = pd.read_excel(join('data', 'state_names.xlsx'))

In [8]:
state_names = list(state_names_df.state.values)

In [9]:
mda_state_counts = []
for mda_file, sentences in mda_sentences.items():
    for i, sentence in enumerate(sentences):
        for state in state_names:
            state_count = sentence.text.lower().count(state.lower())
            mda_state_counts.append([mda_file, i, state, state_count])

In [10]:
mda_state_counts_df = pd.DataFrame(mda_state_counts, columns=['mda_file', 'sen_num', 'state', 'state_count'])

Create dataframe

In [11]:
mda_total_state_counts_df = mda_state_counts_df.groupby(['mda_file', 'state']).state_count.sum().reset_index()
mda_total_state_counts_df.sort_values('state_count', ascending=False).head(20)

,mda_file,state,state_count
673,36146_0001564590-16-013066.txt,Mississippi,50
692,36146_0001564590-16-013066.txt,Texas,38
658,36146_0001564590-16-013066.txt,Florida,30
650,36146_0001564590-16-013066.txt,Alabama,28
745,40533_0000040533-16-000056.txt,Virginia,20
691,36146_0001564590-16-013066.txt,Tennessee,18
808,46250_0000046250-16-000047.txt,Florida,12
54,21510_0000021510-16-000074.txt,California,10
118,21665_0001628280-16-011343.txt,Maine,8
468,31462_0001558370-16-003581.txt,Maine,8


In [12]:
mda_total_state_counts_df.groupby(['state']).state_count.describe().sort_values('mean', ascending=False).head(20)

,count,mean,std,min,25%,50%,75%,max
state,,,,,,,,
Mississippi,20.0,2.5,11.180340,0.0,0.0,0.0,0.0,50.0
Florida,20.0,2.3,7.056912,0.0,0.0,0.0,0.0,30.0
Texas,20.0,2.0,8.485281,0.0,0.0,0.0,0.0,38.0
Maine,20.0,1.7,2.617753,0.0,0.0,0.0,2.0,8.0
Alabama,20.0,1.4,6.260990,0.0,0.0,0.0,0.0,28.0
Virginia,20.0,1.2,4.467426,0.0,0.0,0.0,0.0,20.0
California,20.0,0.9,2.552604,0.0,0.0,0.0,0.0,10.0
Tennessee,20.0,0.9,4.024922,0.0,0.0,0.0,0.0,18.0
New York,20.0,0.6,1.465390,0.0,0.0,0.0,0.0,6.0


### Task 2: Create sentiment score

Follow Loughran and McDonald (2011) and count the number of times a tone word from their dictionary is mentioned.  

Then:  

1. For each MD&A calculate the total number of negative and total number of positive words mentioned.   
2. Tabulate these total counts in a DataFrame together with the total number of words in the MD&A.  
3. Create a new column which calculates a sentiment score using the following equation:  

$$\frac{(Num\ Positive\ Words - Num\ Negative\ Words)}{Total\ Number\ of\ Words}$$



**Note 1:** You can split a sentence into words using any of the tokenizers mentioned in the NLP notebook.  
**Note 2:** The Loughran and McDonald dictionary is included in the "data" folder: `LoughranMcDonald_MasterDictionary_2014.xlsx `

In [13]:
lm_df = pd.read_excel(join('data', 'LoughranMcDonald_MasterDictionary_2014.xlsx'))

In [14]:
negative_words = [str(x).lower() for x in lm_df[lm_df.Negative != 0].Word.values]
positive_words = [str(x).lower() for x in lm_df[lm_df.Positive != 0].Word.values]

In [15]:
mda_sent_counts = []
for mda_file, sentences in tqdm(mda_sentences.items()):
    for i, sentence in enumerate(sentences):
        sen_lower = sentence.text.lower()
        for word in positive_words:
            pos_count = sen_lower.count(word)
            mda_sent_counts.append([mda_file, i, 'pos', word, pos_count])
            
        for word in negative_words:
            neg_count = sen_lower.count(word)
            mda_sent_counts.append([mda_file, i, 'neg', word, neg_count])            

HBox(children=(IntProgress(value=0, max=20), HTML(value='')))

In [16]:
mda_sent_counts_df = pd.DataFrame(mda_sent_counts, columns=['mda_file', 'sen_num', 'tone', 'word', 'count'])

Create DataFrame with total counts:

In [17]:
mda_tone_count_df = mda_sent_counts_df.groupby(['mda_file', 'tone'])['count'].sum().reset_index()

mda_tone_count_df = mda_tone_count_df.pivot(index='mda_file', columns='tone', values='count').reset_index()

mda_tone_count_df.head(2)

tone,mda_file,neg,pos
0,21344_0000021344-16-000050.txt,2318,1166
1,21510_0000021510-16-000074.txt,1078,606


Add total number of words

In [18]:
mda_num_tokens = {}
for key, value in tqdm(mda_data.items()):
    mda_num_tokens[key] = len(nlp(value))

HBox(children=(IntProgress(value=0, max=20), HTML(value='')))

In [19]:
mda_tone_count_df['num_tokens'] = mda_tone_count_df['mda_file'].apply(lambda x: mda_num_tokens[x])

In [20]:
mda_tone_count_df.head(2)

tone,mda_file,neg,pos,num_tokens
0,21344_0000021344-16-000050.txt,2318,1166,53159
1,21510_0000021510-16-000074.txt,1078,606,24899


Create new column with metric

In [21]:
mda_tone_count_df['sentiment_score'] = (mda_tone_count_df['pos'] - mda_tone_count_df['neg']) / mda_tone_count_df['num_tokens']

In [22]:
mda_tone_count_df.head(3)

tone,mda_file,neg,pos,num_tokens,sentiment_score
0,21344_0000021344-16-000050.txt,2318,1166,53159,-0.021671
1,21510_0000021510-16-000074.txt,1078,606,24899,-0.018957
2,21665_0001628280-16-011343.txt,1058,516,34884,-0.015537


### Bonus Task 3: Sentiment score relating to state name references

Count the number of tone words used within a +/- 250 character range for a U.S. state name.

Then:

1. Create a DataFrame for each MD&A where you report the total number of positive and total number of negative words by U.S. state name.  
2. Create aggregate descriptives for the States with the most positive words and the States with most negative words.

In [23]:
def count_terms_within_range(text, state_terms, tone_terms, search_range=150):
    pattern = re.compile('([^\s\w]|_)+')
    text_lower = pattern.sub('', text.lower().replace('\n', ' '))
    
    term_count_state = {state : 0 for state in state_terms}
    
    for term in state_terms:
        matches = [m.span() for m in re.finditer(term.lower() + ' ', text_lower)]
        
        for match in matches:
            sub_text = text_lower[match[0] - search_range : match[1] + search_range].split(' ')
            
            try:
                term_index = sub_text.index(term)
            except Exception as e:
                term_index_list = [i for i, item in enumerate(sub_text) if re.search(term.lower(), item)]    
                    
                if len(term_index_list) > 0:
                    if len(term_index_list) == 1:
                        term_index = term_index_list[0]
                    else: 
                        ## If multiple hits, pick the value that is closest to the center
                        middle = middle = len(term_index_list) / 2
                        dist_list = [abs(i - middle) for i, x in enumerate(term_index_list)]
                        term_index = term_index_list[dist_list.index(min(dist_list))]
                else:
                    term_index = None
                    
            if term_index:  
                lower_bound, upper_bound = term_index - search_range, term_index + search_range + 1
                overlap = set(sub_text[lower_bound: upper_bound]).intersection(tone_terms)
                if overlap:
                    term_count_state[term] += len(overlap)
    return term_count_state

**Note:** My method is very overkill as it is coded to be as fast as possible (at the cost of code readability). Don't worry if your method is different, I just wanted to demonstrate that you can make these things very efficient if you desire so.

In [24]:
mda_range_counts = {}
for key, value in tqdm(mda_data.items()):
    pos_count = count_terms_within_range(value, state_names, set(positive_words), search_range=250)
    neg_count = count_terms_within_range(value, state_names, set(negative_words), search_range=250)
    mda_range_counts[key] = {'pos_count' : pos_count, 'neg_count' : neg_count}

HBox(children=(IntProgress(value=0, max=20), HTML(value='')))

In [25]:
pd.DataFrame(mda_range_counts['36146_0001564590-16-013066.txt']).sort_values('neg_count', ascending=False).head()

,neg_count,pos_count
Mississippi,80,8
Texas,54,10
Florida,48,2
Alabama,28,6
Tennessee,20,6


In [26]:
temp_list = []
for key, value in mda_range_counts.items():
    for tone in ['pos_count', 'neg_count']:
        for k1, v1 in value[tone].items():
            temp_list.append([key, k1, tone, v1])

In [27]:
tone_by_state_df = pd.DataFrame(temp_list, columns=['mda_file', 'state', 'tone', 'count'])

In [28]:
tone_by_state_df.sort_values('count', ascending=False).head()

,mda_file,state,tone,count
1373,36146_0001564590-16-013066.txt,Mississippi,neg_count,80
1392,36146_0001564590-16-013066.txt,Texas,neg_count,54
1358,36146_0001564590-16-013066.txt,Florida,neg_count,48
1350,36146_0001564590-16-013066.txt,Alabama,neg_count,28
1391,36146_0001564590-16-013066.txt,Tennessee,neg_count,20


In [29]:
tone_by_state_agg_df = tone_by_state_df.groupby(['state', 'tone'])['count'].sum().reset_index()

In [30]:
tone_by_state_agg_df[tone_by_state_agg_df.tone == 'pos_count'].sort_values('count', ascending=False).head(5)

,state,tone,count
85,Texas,pos_count,10
47,Mississippi,pos_count,8
1,Alabama,pos_count,6
17,Florida,pos_count,6
83,Tennessee,pos_count,6


In [31]:
tone_by_state_agg_df[tone_by_state_agg_df.tone == 'neg_count'].sort_values('count', ascending=False).head(5)

,state,tone,count
46,Mississippi,neg_count,80
16,Florida,neg_count,54
84,Texas,neg_count,54
0,Alabama,neg_count,28
82,Tennessee,neg_count,20
